In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import gensim
from gensim import corpora, models
import string
from sklearn.feature_extraction.text import CountVectorizer
import ast
from nrclex import NRCLex
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn import svm 
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
from matplotlib.colors import ListedColormap
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import loguniform


In [2]:
emotion_df = pd.read_csv("df_emotions.csv")
emotion_df.head()


,Unnamed: 0,text,class,stemmed_processed_text,lemmatized_processed_text,clean_lemmatized_processed_text,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,anticipation
0,0,ex wife threatening suicide recently i left my...,1,"['ex', 'wife', 'threaten', 'suicid', 'recent',...","['ex', 'wife', 'threatening', 'suicide', 'rece...",ex wife threatening suicide recently left wife...,0.125000,0.125000,0.0,0.078125,0.078125,0.078125,0.171875,0.093750,0.062500,0.078125,0.109375
1,1,am i weird i do not get affected by compliment...,0,"['weird', 'not', 'get', 'affect', 'compliment'...","['weird', 'not', 'get', 'affected', 'complimen...",weird not get affected compliment coming someo...,0.066667,0.000000,0.0,0.133333,0.133333,0.133333,0.133333,0.000000,0.066667,0.133333,0.200000
2,2,finally is almost over so i can never hear has...,0,"['final', 'almost', 'never', 'hear', 'bad', 'y...","['finally', 'almost', 'never', 'hear', 'bad', ...",finally almost never hear bad year ever swear ...,0.100000,0.100000,0.0,0.150000,0.050000,0.150000,0.100000,0.050000,0.100000,0.100000,0.100000
3,3,i need help just help me i am crying so hard,1,"['need', 'help', 'help', 'cri', 'hard']","['need', 'help', 'help', 'cry', 'hard']",need help help cry hard,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000
4,4,i m so lost hello my name is adam and i ve bee...,1,"['lost', 'hello', 'name', 'adam', 'struggl', '...","['lost', 'hello', 'name', 'adam', 'struggling'...",lost hello name adam struggling year afraid pa...,0.146226,0.103774,0.0,0.066038,0.037736,0.075472,0.198113,0.146226,0.061321,0.042453,0.122642


In [3]:
emotion_df.drop(columns = ["text","lemmatized_processed_text","class","stemmed_processed_text","Unnamed: 0"],axis=1,inplace=True)
emotion_df.head()

,clean_lemmatized_processed_text,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,anticipation
0,ex wife threatening suicide recently left wife...,0.125000,0.125000,0.0,0.078125,0.078125,0.078125,0.171875,0.093750,0.062500,0.078125,0.109375
1,weird not get affected compliment coming someo...,0.066667,0.000000,0.0,0.133333,0.133333,0.133333,0.133333,0.000000,0.066667,0.133333,0.200000
2,finally almost never hear bad year ever swear ...,0.100000,0.100000,0.0,0.150000,0.050000,0.150000,0.100000,0.050000,0.100000,0.100000,0.100000
3,need help help cry hard,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000
4,lost hello name adam struggling year afraid pa...,0.146226,0.103774,0.0,0.066038,0.037736,0.075472,0.198113,0.146226,0.061321,0.042453,0.122642


In [4]:
emotion_df_2 = pd.read_csv("lexicon_approach2.csv")
emotion_df_2.head()


,Unnamed: 0,text,class,stemmed_processed_text,lemmatized_processed_text,clean_lemmatized_processed_text,neg,neu,pos,compound
0,0,ex wife threatening suicide recently i left my...,suicide,"['ex', 'wife', 'threaten', 'suicid', 'recent',...","['ex', 'wife', 'threatening', 'suicide', 'rece...",ex wife threatening suicide recently left wife...,0.378,0.437,0.186,-0.9655
1,1,am i weird i do not get affected by compliment...,non-suicide,"['weird', 'not', 'get', 'affect', 'compliment'...","['weird', 'not', 'get', 'affected', 'complimen...",weird not get affected compliment coming someo...,0.225,0.529,0.245,0.0984
2,2,finally is almost over so i can never hear has...,non-suicide,"['final', 'almost', 'never', 'hear', 'bad', 'y...","['finally', 'almost', 'never', 'hear', 'bad', ...",finally almost never hear bad year ever swear ...,0.259,0.433,0.308,0.2025
3,3,i need help just help me i am crying so hard,suicide,"['need', 'help', 'help', 'cri', 'hard']","['need', 'help', 'help', 'cry', 'hard']",need help help cry hard,0.413,0.092,0.495,0.2263
4,4,i m so lost hello my name is adam and i ve bee...,suicide,"['lost', 'hello', 'name', 'adam', 'struggl', '...","['lost', 'hello', 'name', 'adam', 'struggling'...",lost hello name adam struggling year afraid pa...,0.368,0.505,0.127,-0.9965


In [5]:
emotion_df_2.drop(columns = ["text","lemmatized_processed_text","stemmed_processed_text","Unnamed: 0","clean_lemmatized_processed_text"],axis=1,inplace=True)
emotion_df_2.head()

,class,neg,neu,pos,compound
0,suicide,0.378,0.437,0.186,-0.9655
1,non-suicide,0.225,0.529,0.245,0.0984
2,non-suicide,0.259,0.433,0.308,0.2025
3,suicide,0.413,0.092,0.495,0.2263
4,suicide,0.368,0.505,0.127,-0.9965


In [6]:
combined_df = pd.concat([emotion_df, emotion_df_2], axis=1)
combined_df.dropna(inplace=True)
combined_df.head()

,clean_lemmatized_processed_text,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,anticipation,class,neg,neu,pos,compound
0,ex wife threatening suicide recently left wife...,0.125000,0.125000,0.0,0.078125,0.078125,0.078125,0.171875,0.093750,0.062500,0.078125,0.109375,suicide,0.378,0.437,0.186,-0.9655
1,weird not get affected compliment coming someo...,0.066667,0.000000,0.0,0.133333,0.133333,0.133333,0.133333,0.000000,0.066667,0.133333,0.200000,non-suicide,0.225,0.529,0.245,0.0984
2,finally almost never hear bad year ever swear ...,0.100000,0.100000,0.0,0.150000,0.050000,0.150000,0.100000,0.050000,0.100000,0.100000,0.100000,non-suicide,0.259,0.433,0.308,0.2025
3,need help help cry hard,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,suicide,0.413,0.092,0.495,0.2263
4,lost hello name adam struggling year afraid pa...,0.146226,0.103774,0.0,0.066038,0.037736,0.075472,0.198113,0.146226,0.061321,0.042453,0.122642,suicide,0.368,0.505,0.127,-0.9965


In [7]:
## Normalizing Data as naive bayes dont accept negative
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
combined_df['norm_compound'] = scaler.fit_transform(combined_df[['compound']])
combined_df.head()

,clean_lemmatized_processed_text,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,anticipation,class,neg,neu,pos,compound,norm_compound
0,ex wife threatening suicide recently left wife...,0.125000,0.125000,0.0,0.078125,0.078125,0.078125,0.171875,0.093750,0.062500,0.078125,0.109375,suicide,0.378,0.437,0.186,-0.9655,0.01725
1,weird not get affected compliment coming someo...,0.066667,0.000000,0.0,0.133333,0.133333,0.133333,0.133333,0.000000,0.066667,0.133333,0.200000,non-suicide,0.225,0.529,0.245,0.0984,0.54920
2,finally almost never hear bad year ever swear ...,0.100000,0.100000,0.0,0.150000,0.050000,0.150000,0.100000,0.050000,0.100000,0.100000,0.100000,non-suicide,0.259,0.433,0.308,0.2025,0.60125
3,need help help cry hard,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,suicide,0.413,0.092,0.495,0.2263,0.61315
4,lost hello name adam struggling year afraid pa...,0.146226,0.103774,0.0,0.066038,0.037736,0.075472,0.198113,0.146226,0.061321,0.042453,0.122642,suicide,0.368,0.505,0.127,-0.9965,0.00175


### TESTING 3 VECTORS

#### NRC FLEX

In [8]:
import scipy.sparse as sp

# Create a sparse matrix for the extra features
extra_features_sparse = sp.csr_matrix(combined_df[["fear", "anger", "anticip", "trust", "surprise", "positive", "negative", "sadness","disgust", "joy", "anticipation"]].values)

# Fit and transform the TF-IDF matrix
tfidf_vectorizer_sa_nrc = TfidfVectorizer()
tfidf_sa_nrc = tfidf_vectorizer_sa_nrc.fit_transform(combined_df.clean_lemmatized_processed_text)

# Combine the two sparse matrices using hstack
all_features_sa_nrc = sp.hstack([tfidf_sa_nrc, extra_features_sparse])

In [9]:
train_X_sa_nrc, test_X_sa_nrc, train_y_sa_nrc, test_y_sa_nrc = train_test_split(all_features_sa_nrc,combined_df['class'],test_size=0.3, random_state = 42)

In [10]:
print("n_samples: %d, n_features: %d" % train_X_sa_nrc.shape)
print("n_samples: %d, n_features: %d" % test_X_sa_nrc.shape)

n_samples: 162385, n_features: 47565
n_samples: 69594, n_features: 47565


#### VADER1  (Pos, neg, neu, comp)

In [11]:
extra_features_sparse = sp.csr_matrix(combined_df[["pos","neg","neu","norm_compound"]].values)

# Fit and transform the TF-IDF matrix
tfidf_vectorizer_sa_va_1 = TfidfVectorizer()
tfidf_sa_va_1 = tfidf_vectorizer_sa_va_1.fit_transform(combined_df.clean_lemmatized_processed_text)

# Combine the two sparse matrices using hstack
all_features_sa_va_1 = sp.hstack([tfidf_sa_va_1, extra_features_sparse])

In [12]:
train_X_sa_va_1, test_X_sa_va_1, train_y_sa_va_1, test_y_sa_va_1 = train_test_split(all_features_sa_va_1,combined_df['class'],test_size=0.3, random_state = 42)

In [13]:
print("n_samples: %d, n_features: %d" % train_X_sa_va_1.shape)
print("n_samples: %d, n_features: %d" % test_X_sa_va_1.shape)

n_samples: 162385, n_features: 47558
n_samples: 69594, n_features: 47558


#### VADER2  (comp)

In [14]:
extra_features_sparse = sp.csr_matrix(combined_df[["norm_compound"]].values)

# Fit and transform the TF-IDF matrix
tfidf_vectorizer_sa_va_2 = TfidfVectorizer()
tfidf_sa_va_2 = tfidf_vectorizer_sa_va_2.fit_transform(combined_df.clean_lemmatized_processed_text)

# Combine the two sparse matrices using hstack
all_features_sa_va_2 = sp.hstack([tfidf_sa_va_2, extra_features_sparse])

In [15]:
train_X_sa_va_2, test_X_sa_va_2, train_y_sa_va_2, test_y_sa_va_2 = train_test_split(all_features_sa_va_2,combined_df['class'],test_size=0.3, random_state = 42)

In [16]:
print("n_samples: %d, n_features: %d" % train_X_sa_va_2.shape)
print("n_samples: %d, n_features: %d" % test_X_sa_va_2.shape)

n_samples: 162385, n_features: 47555
n_samples: 69594, n_features: 47555


## Naive Bayes

#### NRC FLEX

In [17]:
#Naive Bayes Classifier
naive_bayes_classifier_sa_nrc = MultinomialNB()
naive_bayes_classifier_sa_nrc.fit(train_X_sa_nrc, train_y_sa_nrc)
#predicted y
y_pred_nb_sa_nrc = naive_bayes_classifier_sa_nrc.predict(test_X_sa_nrc)

In [18]:
nb_accuracy_sa_nrc = accuracy_score(y_pred_nb_sa_nrc, test_y_sa_nrc)
print("Naive Bayes Accuracy Score -> ", nb_accuracy_sa_nrc * 100)

Naive Bayes Accuracy Score ->  89.67008650170992


#### VADER1  (Pos, neg, neu, comp)

In [19]:
#Naive Bayes Classifier
naive_bayes_classifier_sa_va_1 = MultinomialNB()
naive_bayes_classifier_sa_va_1.fit(train_X_sa_va_1, train_y_sa_va_1)
#predicted y
y_pred_nb_sa_va_1 = naive_bayes_classifier_sa_va_1.predict(test_X_sa_va_1)

In [20]:
nb_accuracy_sa_va_1 = accuracy_score(y_pred_nb_sa_va_1, test_y_sa_va_1)
print("Naive Bayes Accuracy Score -> ", nb_accuracy_sa_va_1 * 100)

Naive Bayes Accuracy Score ->  90.46613213782797


#### VADER1  (comp)

In [21]:
#Naive Bayes Classifier
naive_bayes_classifier_sa_va_2 = MultinomialNB()
naive_bayes_classifier_sa_va_2.fit(train_X_sa_va_2, train_y_sa_va_2)
#predicted y
y_pred_nb_sa_va_2 = naive_bayes_classifier_sa_va_2.predict(test_X_sa_va_2)

In [22]:
nb_accuracy_sa_va_2 = accuracy_score(y_pred_nb_sa_va_2, test_y_sa_va_2)
print("Naive Bayes Accuracy Score -> ", nb_accuracy_sa_va_2 * 100)

Naive Bayes Accuracy Score ->  90.08535218553324


## Logistic Regression

#### NRC FLEX

In [23]:
# fit the training dataset on the Logistic Regression classifier
logreg_sa_nrc = LogisticRegression(max_iter=200)
logreg_sa_nrc.fit(train_X_sa_nrc,train_y_sa_nrc)
# predict the labels on validation dataset
y_pred_logreg_sa_nrc = logreg_sa_nrc.predict(test_X_sa_nrc)

In [24]:
logreg_accuracy_sa_nrc = accuracy_score(y_pred_logreg_sa_nrc, test_y_sa_nrc)
print("Logistic Regression Accuracy Score -> ", logreg_accuracy_sa_nrc * 100)

Logistic Regression Accuracy Score ->  93.52961462195016


#### VADER1  (Pos, neg, neu, comp)

In [25]:
# fit the training dataset on the Logistic Regression classifier
logreg_sa_va_1 = LogisticRegression(max_iter=200)
logreg_sa_va_1.fit(train_X_sa_va_1,train_y_sa_va_1)
# predict the labels on validation dataset
y_pred_logreg_sa_va_1 = logreg_sa_va_1.predict(test_X_sa_va_1)

In [26]:
logreg_accuracy_sa_va_1 = accuracy_score(y_pred_logreg_sa_va_1, test_y_sa_va_1)
print("Logistic Regression Accuracy Score -> ", logreg_accuracy_sa_va_1 * 100)

Logistic Regression Accuracy Score ->  93.54685748771446


#### VADER1  (comp)

In [27]:
# fit the training dataset on the Logistic Regression classifier
logreg_sa_va_2 = LogisticRegression(max_iter=200)
logreg_sa_va_2.fit(train_X_sa_va_2,train_y_sa_va_2)
# predict the labels on validation dataset
y_pred_logreg_sa_va_2 = logreg_sa_va_2.predict(test_X_sa_va_2)

In [28]:
logreg_accuracy_sa_va_2 = accuracy_score(y_pred_logreg_sa_va_2, test_y_sa_va_2)
print("Logistic Regression Accuracy Score -> ", logreg_accuracy_sa_va_2 * 100)

Logistic Regression Accuracy Score ->  93.52961462195016


## SVM

#### NRC FLEX

In [29]:
# fit the training dataset on the SVM classifier
linearSVC_sa_nrc = LinearSVC()
linearSVC_sa_nrc.fit(train_X_sa_nrc,train_y_sa_nrc)
# predict the labels on validation dataset
y_pred_SVC_sa_nrc = linearSVC_sa_nrc.predict(test_X_sa_nrc)


In [30]:
svc_accuracy_sa_nrc = accuracy_score(y_pred_SVC_sa_nrc, test_y_sa_nrc)
print("SVM Accuracy Score -> ",svc_accuracy_sa_nrc*100)

SVM Accuracy Score ->  93.45489553697158


#### VADER1  (Pos, neg, neu, comp)

In [31]:
# fit the training dataset on the SVM classifier
linearSVC_sa_va_1 = LinearSVC()
linearSVC_sa_va_1.fit(train_X_sa_va_1,train_y_sa_va_1)
# predict the labels on validation dataset
y_pred_SVC_sa_va_1 = linearSVC_sa_va_1.predict(test_X_sa_va_1)


In [32]:
svc_accuracy_sa_va_1 = accuracy_score(y_pred_SVC_sa_va_1, test_y_sa_va_1)
print("SVM Accuracy Score -> ",svc_accuracy_sa_va_1*100)

SVM Accuracy Score ->  93.47070149725552


#### VADER1  (comp)

In [33]:
# fit the training dataset on the SVM classifier
linearSVC_sa_va_2 = LinearSVC()
linearSVC_sa_va_2.fit(train_X_sa_va_2,train_y_sa_va_2)
# predict the labels on validation dataset
y_pred_SVC_sa_va_2 = linearSVC_sa_va_2.predict(test_X_sa_va_2)


In [34]:
svc_accuracy_sa_va_2 = accuracy_score(y_pred_SVC_sa_va_2, test_y_sa_va_2)
print("SVM Accuracy Score -> ",svc_accuracy_sa_va_2*100)

SVM Accuracy Score ->  93.4678276862948
